In [11]:
import Pkg; 

if split(pwd(),"/")[end] == "random_walk"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using GLMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances

In [12]:
mol_type = "6r7m"
template_centers = MorphoMol.Utilities.TMV_TEMPLATES[mol_type]["template_centers"]
template_radii = MorphoMol.Utilities.TMV_TEMPLATES[mol_type]["template_radii"]
n_mol = 2
bounds = 80.0
x_init = MorphoMol.Utilities.get_initial_state(n_mol, bounds)
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);

In [ ]:
T = 0.65
β = 1.0 / T

#persistence_weights = [0.5, -0.25, -0.1]
persistence_weights = [0.0, 1.0, 0.0]

σ_r = 0.15
σ_t = 1.25
Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

input = Dict(
    "template_centers" => template_centers,
    "template_radii" => template_radii,
    "n_mol" => n_mol,
    "σ_r" => σ_r,
    "σ_t" => σ_t,
    "persistence_weights" => persistence_weights,
    "T" => T,
    "mol_type" => mol_type
)

#energy(x) = solvation_free_energy_and_measures(x,template_centers, radii, rs, pf, 0.0, overlap_slope, 1.0)
energy(x) = persistence(x, template_centers, persistence_weights)
perturbation(x) = perturb_single_randomly_chosen(x, σ_r, σ_t)

output = Dict{String, Vector}(
    "states" => Vector{Vector{Float64}}([]),
    "Es" => Vector{Float64}([]),
    "αs" => Vector{Float32}([]),
    "P0s" => Vector{Float64}([]),
    "P1s" => Vector{Float64}([]),
    "P2s" => Vector{Float64}([]),
)

rwm = MorphoMol.Algorithms.RandomWalkMetropolis(energy, perturbation, β)
simulation_time_minutes = 10.0

10.0

In [38]:
MorphoMol.Algorithms.simulate!(rwm, x_init, simulation_time_minutes, output);
output["αs"][end]

0.61310345f0

In [39]:
length(output["states"])

1779

In [ ]:
n = length(output["Es"])
e_mindex = argmin(output["Es"])

figure_config = Dict(
    "plot_ms" => 5,
    "tracker_ms" => 10,
    "title_fs" => 20,
    "vis_range" =>  maximum([1, e_mindex-750]):minimum([e_mindex + 50, n])
)

f = Figure(fontsize = 12)
#Slider
sl_i = Slider(f[3, 1:3], range = 1:length(output["states"][figure_config["vis_range"]]), startvalue = 1)
x = sl_i.value
cgl = GridLayout(f[1,1])
MorphoMolNotebooks.visualize_configuration_sequence!(cgl, x, input, output, figure_config)

aspgl = GridLayout(f[2, 1:2])    
MorphoMolNotebooks.visualize_alpha_shape_persistence_measures!(aspgl, x, input, output, figure_config)

etgl = GridLayout(f[2, 3])
MorphoMolNotebooks.visualize_energy_and_theta!(etgl, x, input, output, figure_config)

display(f)

GLMakie.Screen(...)

: 